**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [28]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [29]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [30]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-10 15:05:21,236 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-10 15:05:21.238 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [31]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-10 15:05:21.336 | INFO     | __main__:<module>:13 - User Role: reviewer
2025-02-10 15:05:21.338 | INFO     | __main__:<module>:24 - Creating account with name: optimistic_aryabhata
2025-02-10 15:05:21.340 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:05:21.358 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-10 15:05:21.360 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:05:21.361 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:05:21.362 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"


2025-02-10 15:05:21,378 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:05:21,380 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:05:21,531 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:05:21,614 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:05:21.618 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'logs/accounts.json', current total entries: 2
2025-02-10 15:05:21.620 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


3 - Queries the blockchain to confirm the succesful creation of the account.

In [32]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-10 15:05:21.636 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:05:21,639 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:05:21,657 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:05:21,659 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:05:24,637 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:05:24,646 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:05:24,648 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'c36833ab556c05bcee27264d0e1465d25e4f556ea217feb4bed614d32c90edef', 'timestamp': 1739199924648}
2025-02-10 15:05:24.650 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:05:24,654 - INFO - Appended data under 'optimistic_aryabhata@test' in logs/account_data.json
2025-02-10 15:05:24.655 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [33]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-10 15:05:24,702 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-10 15:05:24.704 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [34]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-10 15:05:24.763 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:05:24.785 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-10 15:05:24.786 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-10 15:05:24.787 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:05:24.788 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-10 15:05:24,790 - INFO - Processing account: optimistic_aryabhata@test
2025-02-10 15:05:24,791 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Optimistic Aryabhata', 'foaf:mbox': 'optimistic_aryabhata@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Muthesius-Hochschule

2025-02-10 15:05:24.825 | SUCCESS  | ipfs_functions:upload_json_to_ipfs:42 - Success uploading JSON to IPFS. CID: QmUFhDpzQi3gXhvqPjUfWrrjCyoTQarBC6dXuxFP7fEiTu
2025-02-10 15:05:24,826 - INFO - User Account Metadata CID: QmUFhDpzQi3gXhvqPjUfWrrjCyoTQarBC6dXuxFP7fEiTu
2025-02-10 15:05:24.827 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-10 15:05:24,843 - INFO - None
2025-02-10 15:05:24,846 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:05:27,670 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:05:27,679 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:05:27.681 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-10 15:05:27.701 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000

6 - Queries the blockchain again to confirm the succesfull details setting.

In [35]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details)

2025-02-10 15:05:27.725 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:05:27,727 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:05:27,744 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:05:27,745 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:05:30,704 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:05:30,713 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:05:30,715 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'c274d30f98e13f998235692de21b051b51c8db8177f2fdc1dec28291f71e6494', 'timestamp': 1739199930715}
2025-02-10 15:05:30.716 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:05:30,719 - INFO - Appended data under 'optimistic_aryabhata@test' in logs/account_data.json
2025-02-10 15:05:30.720 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [36]:
user_metadata = download_json_from_ipfs(account_metadata_cid)


2025-02-10 15:05:30.778 | SUCCESS  | ipfs_functions:download_json_from_ipfs:52 - Success downloading JSON from IPFS. Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'optimistic_aryabhata@test', 'schema:publicKey': '580b63a70112040e38248f0430798ebd77ace027ff00e4757f411d7c30590c78', 'schema:roleName': 'reviewer'}, 'foaf:mbox': 'optimistic_aryabhata@email.com', 'foaf:name': 'Optimistic Aryabhata', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Muthesius-Hochschule, Fachhochschule für Kunst und Gestaltung'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '4138-5255-4792-X'}}
